# Assignment 3 Top-Level Code/Notebook
### Training a language model base on Karpathy's minGPT codebase


In [1]:
import torch 
import numpy as np

from nltk.tokenize import sent_tokenize 

from pathlib import Path 
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from mingpt.bpe import BPETokenizer 
from mingpt.utils import set_seed 
set_seed(1234)

In [2]:
"""
Prepare the dataset to train the Language Model (LM)
This implementation splits the sentences and so doesn't create training 
examples that cross sentences.

This code is set so that it uses one of two possible datasets, which were also used in Assignment 1: 
SmallSimpleCorpus.txt or LargerCorpus.txt

Arguments:
            ds_choice: str. "small" or "large". (i.e. selects which of the two datasets)
            split: str. "train" or "test".
            truncation: int. If -1: no truncation on sentences. Otherwise: truncate to this specific length.
""" 

class LanguageModelingDataset(Dataset):
    
    def __init__(self, ds_choice="small", split="train", truncation=-1):
        
        base_path = "./"
        fn = {"small": "SmallSimpleCorpus.txt", "large": "LargerCorpus.txt"}
        self.ds_choice = ds_choice
        self.truncation = truncation  # int. If -1, then
        text = Path(base_path, fn[ds_choice]).read_text()
        if ds_choice == "large":
            # Remove the newline char in the middle of sentences
            # The "paragraph splitting" newlines appear to be \n\n -- remove the duplications there
            text = text.replace("\n\n", "$$^^$$").replace("\n", " ").replace("$$^^$$", "\n")
        sentences = sent_tokenize(text)

        # Train / test split
        train, val = train_test_split(sentences, test_size=0.2, shuffle=False)
        if split == "train":
            raw_data = train 
        else:
            raw_data = val 

        # Tokenize
        self.tokenizer = BPETokenizer()
        self.data = []  # List of 1-d pytorch tensor
        for sent in raw_data:
            tokenized = self.tokenizer(sent).view(-1)  # pytorch tensor
            if truncation >= 0:
                self.data.append(tokenized[:truncation])
            else:
                self.data.append(tokenized)

        # Count some items
        self.max_sentence_length = np.max([len(d) for d in self.data])

    def __len__(self):
        return len(self.data)

    def get_vocab_size(self):
        """
        We have to set this to the max vocab size (i.e., that decided by the BPE tokenizer), 
        but actually, only a small number of vocab is used, especially for the small text. 
        """
        return 50257

    def __getitem__(self, idx):
        """
        The output should be a tuple x and y, both as pytorch tensors.
        Please refer to the `run()` method in the mingpt/trainer.py script for 
        how the x and y are going to be used.
        """
        x = self.data[idx][:-1]
        y = self.data[idx][1:]
        return (x, y)

    def get_block_size(self):
        """
        block_size is the size at which lines are truncated to ensure they are equal-length.
        """
        return self.max_sentence_length
    
# Instantiate the Training Dataset
train_dataset = LanguageModelingDataset(ds_choice="small", split="train")  # use this for the short corpus
#train_dataset = LanguageModelingDataset(ds_choice="large", split="train", truncation=512) #use this for long

# Instantiate a Validation Dataset (this is only really needed for the fine-tune task, not the LM task)
val_dataset = LanguageModelingDataset(ds_choice="small", split="validation")
#val_dataset = LanguageModelingDataset(ds_choice="large", split="validation", truncation=512)

In [3]:
def lm_collate_fn(batch, device):
    x = [item[0] for item in batch]  # List (len B) of varying lengths
    y = [item[1] for item in batch]  # List (len B) of the same lengths as x
    maxlen = max([len(s) for s in x])

    padded_x, padded_y = [], []
    for sx, sy in zip(x, y):
        padded_x.append(torch.cat([sx, torch.ones(maxlen - len(sx))]))
        padded_y.append(torch.cat([sy, torch.ones(maxlen - len(sy))]))
    return torch.stack(padded_x).long().to(device), torch.stack(padded_y).long().to(device)


In [4]:
# Print out an example of the data - this is processed more once it reaches lm_collate_fn (above)
x,y = train_dataset[5]
print(x, y)
print("X: ",train_dataset.tokenizer.decode(x))
print("Y: ",train_dataset.tokenizer.decode(y))

tensor([  40, 6437,  262, 3290]) tensor([6437,  262, 3290,   13])
X:  I rub the dog
Y:   rub the dog.


In [5]:
from mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt-nano'
model_config.vocab_size = train_dataset.get_vocab_size()
model_config.block_size = train_dataset.get_block_size()
model_config.n_classification_class = 2
model = GPT(model_config)

number of parameters: 2.50M


In [6]:
torch.cuda.is_available()

True

In [7]:
# Create a Trainer object and set the core hyper-parameters
from mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4 # the model we're using is so small that we can go a bit faster
train_config.max_iters = 3000  # For small corpus: 3000 iterations is plenty. For large corpus: 100000 iterations is needed
train_config.num_workers = 0
train_config.batch_size = 4    # For small corpus, batch size of 4 is fine.  For large corpus use 16
trainer = Trainer(train_config, model, train_dataset, val_dataset, collate_fn=lm_collate_fn)

running on device cuda


In [8]:
# This function is called at the end of every batch in training
# and is used to report the amount of time per 100 batches, and the loss at that point

def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
trainer.set_callback('on_batch_end', batch_end_callback)

# Train!
trainer.run()

iter_dt 0.00ms; iter 0: train loss 10.81249
iter_dt 19.00ms; iter 100: train loss 5.96995
iter_dt 19.00ms; iter 200: train loss 2.49559
iter_dt 22.00ms; iter 300: train loss 1.49280
iter_dt 19.00ms; iter 400: train loss 0.83902
iter_dt 18.00ms; iter 500: train loss 0.78918
iter_dt 19.00ms; iter 600: train loss 0.83952
iter_dt 19.00ms; iter 700: train loss 0.70429
iter_dt 21.00ms; iter 800: train loss 0.64494
iter_dt 19.01ms; iter 900: train loss 0.59071
iter_dt 18.00ms; iter 1000: train loss 0.56029
iter_dt 19.00ms; iter 1100: train loss 0.76987
iter_dt 19.00ms; iter 1200: train loss 0.58646
iter_dt 19.02ms; iter 1300: train loss 0.61791
iter_dt 19.00ms; iter 1400: train loss 0.66156
iter_dt 19.00ms; iter 1500: train loss 0.68874
iter_dt 19.01ms; iter 1600: train loss 0.69681
iter_dt 19.00ms; iter 1700: train loss 0.62078
iter_dt 19.09ms; iter 1800: train loss 0.58298
iter_dt 18.34ms; iter 1900: train loss 0.59302
iter_dt 19.02ms; iter 2000: train loss 0.59085
iter_dt 19.01ms; iter 210

In [9]:
model.to(trainer.device)
# store the saved model in a file, so can re-use later
modelsavename= "model_filename.pt"  # change the name here to save in a specific file (and restore below)
with open(modelsavename, "wb") as f:
    torch.save(trainer.model.state_dict(), f)

In [10]:
# Use the trained language model to predict a sequence of words following a few words
encoded_prompt = train_dataset.tokenizer("He and I").to(trainer.device)
generated_sequence, idx, probs = trainer.model.modified_generate(encoded_prompt, trainer.device, temperature=0.8, max_new_tokens=10)

In [11]:
train_dataset.tokenizer.decode(generated_sequence[0])

'He and I hold the dog.. cat. cat and dog'

In [31]:
word = []
for i, x in enumerate(idx):
    col = []
    for j, y in enumerate(x):
        col.append(train_dataset.tokenizer.decode(idx[i][j].reshape(1)) + f"  {probs[i][j]:.3f}")
    word.append(col)
    
import pandas as pd
result = pd.DataFrame(data=word)
result.T

,0,1,2,3,4,5,6,7,8,9
0,hold 0.491,the 0.525,dog 0.564,. 0.996,. 0.953,cat 0.772,. 0.841,cat 0.720,and 0.647,dog 0.976
1,rub 0.262,a 0.472,cat 0.435,. 0.003,. 0.013,dog 0.224,and 0.156,dog 0.269,. 0.350,cat 0.013
2,can 0.246,and 0.002,a 0.000,and 0.000,cat 0.013,a 0.001,a 0.002,and 0.005,can 0.001,rub 0.008
3,holds 0.001,cat 0.000,. 0.000,cat 0.000,dog 0.012,the 0.001,the 0.001,a 0.003,rub 0.001,can 0.001
4,and 0.000,hold 0.000,and 0.000,rub 0.000,a 0.004,and 0.001,. 0.000,the 0.002,holds 0.001,holds 0.001
5,cat 0.000,holds 0.000,the 0.000,dog 0.000,the 0.003,hold 0.000,cat 0.000,. 0.001,a 0.000,hold 0.000


In [22]:
train_dataset.tokenizer.decode(generated_sequence[0][0].reshape(1))

'He'

In [32]:
# Another example
encoded_prompt = train_dataset.tokenizer("She rubs").to(trainer.device)
generated_sequence, idx, probs = trainer.model.modified_generate(encoded_prompt, trainer.device, temperature=0.6, max_new_tokens=10)


In [33]:
train_dataset.tokenizer.decode(generated_sequence[0])

'She rubs a cat and dog. cat. cat. cat'

In [34]:
word = []
for i, x in enumerate(idx):
    col = []
    for j, y in enumerate(x):
        col.append(train_dataset.tokenizer.decode(idx[i][j].reshape(1)) + f"  {probs[i][j]:.3f}")
    word.append(col)
    
import pandas as pd
result = pd.DataFrame(data=word)
result.T

,0,1,2,3,4,5,6,7,8,9
0,a 0.442,cat 0.535,and 0.665,dog 0.999,. 0.999,cat 0.683,. 0.551,cat 0.684,. 0.992,cat 0.607
1,the 0.370,dog 0.465,. 0.335,cat 0.001,. 0.000,dog 0.317,and 0.435,dog 0.315,. 0.008,dog 0.393
2,and 0.188,holds 0.000,a 0.000,holds 0.000,and 0.000,a 0.000,can 0.006,and 0.000,cat 0.000,a 0.000
3,. 0.000,. 0.000,the 0.000,rub 0.000,rub 0.000,and 0.000,holds 0.005,a 0.000,a 0.000,the 0.000
4,cat 0.000,. 0.000,can 0.000,can 0.000,a 0.000,the 0.000,rub 0.003,the 0.000,the 0.000,and 0.000
5,holds 0.000,and 0.000,holds 0.000,a 0.000,the 0.000,s 0.000,I 0.000,. 0.000,and 0.000,s 0.000


In [16]:
torch.round(probs, decimals=3)

tensor([[1.0000, 1.0000, 1.0000, 0.4790, 0.5130, 0.7420, 1.0000, 1.0000, 0.8940,
         0.5980, 1.0000, 0.9990, 0.9730]], device='cuda:0')

In [ ]:
# The code below shows how to reload the model from the saved file; is useful things that take long to train
model.load_state_dict(torch.load(modelsavename))

In [17]:
# Example showing how the reloaded model still works
encoded_prompt = train_dataset.tokenizer("I hold").to(trainer.device)
generated_sequence, probs = trainer.model.modified_generate(encoded_prompt, trainer.device, temperature=0.6, max_new_tokens=10)


In [18]:
train_dataset.tokenizer.decode(generated_sequence[0])

'I hold a cat and dog.. cat. cat.'

In [19]:
torch.round(probs, decimals=3)

tensor([[1.0000, 1.0000, 0.3860, 0.5210, 0.7890, 0.9990, 1.0000, 0.8280, 0.6710,
         0.9970, 0.6310, 0.9970]], device='cuda:0')